# Analysis
This code contains all of the analysis for data processing and creating the 6 tables detailed below.

## Importing Libraries

In [5]:
import pandas as pd
import copy

## Loading the data

In [26]:
data = pd.read_csv("../data_clean/wp_politicians_by_country.csv")

## Table 1: Top 10 countries by coverage

In [40]:
# Compute the articles per capita
coverage_data = copy.deepcopy(data)
coverage_data = coverage_data.groupby('country').agg( Count = ('population', 'count'), population = ('population', 'mean')).reset_index()
coverage_data['articles_per_capita'] = coverage_data["Count"] / coverage_data["population"]

#Print the Top 10 to a Table
print(coverage_data.sort_values(by='articles_per_capita', ascending=False).head(10)[['country', 'articles_per_capita']])

                            country  articles_per_capita
97                           Monaco                  inf
155                          Tuvalu                  inf
4               Antigua and Barbuda           330.000000
51   Federated States of Micronesia           140.000000
94                 Marshall Islands           130.000000
150                           Tonga           100.000000
12                         Barbados            83.333333
126                      Seychelles            60.000000
99                       Montenegro            56.666667
17                           Bhutan            55.000000


## Table 2: Bottom 10 countries by coverage

In [41]:
#Print the Top 10 to a Table
print(coverage_data.sort_values(by='articles_per_capita').head(10)[['country', 'articles_per_capita']])

           country  articles_per_capita
31           China             0.011337
57           Ghana             0.087977
66           India             0.105698
123   Saudi Arabia             0.135501
165         Zambia             0.148515
109         Norway             0.181818
70          Israel             0.204082
45           Egypt             0.304183
37   Cote d'Ivoire             0.323625
50        Ethiopia             0.347826


## Table 3: Top 10 countries by high quality

In [ ]:
af

## Table 4: Bottom 10 countries by high quality

## Table 5: Geographic regions by total coverage

## Table 6: Geographic regions by high quality coverage